Passo a passo utilizado para criação da pilha (stack) 

https://docs.aws.amazon.com/pt_br/AWSCloudFormation/latest/UserGuide/gettingstarted.walkthrough.html


Criar sua primeira pilha:

Para criar uma pilha Hello world do CloudFormation com o console
Abra o console do CloudFormation.

Selecione Create Stack (Criar pilha).

Na página Criar pilha, escolha Compilar do Infrastructure Composer e depois Criar no Infrastructure Composer. Isso leva você ao Infrastructure Composer no modo de console do CloudFormation, onde você pode carregar e validar o exemplo de modelo.

Para carregar e validar o modelo de exemplo, faça o seguinte:

Escolha Modelos. Depois, copie e cole o seguinte modelo do CloudFormation no editor de modelos:



In [ ]:
AWSTemplateFormatVersion: 2010-09-09
Description: CloudFormation Template for WebServer with Security Group and EC2 Instance

Parameters:
  LatestAmiId:
    Description: The latest Amazon Linux 2 AMI from the Parameter Store
    Type: 'AWS::SSM::Parameter::Value<AWS::EC2::Image::Id>'
    Default: '/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'

  InstanceType:
    Description: WebServer EC2 instance type
    Type: String
    Default: t2.micro
    AllowedValues:
      - t3.micro
      - t2.micro
    ConstraintDescription: must be a valid EC2 instance type.
    
  MyIP:
    Description: Your IP address in CIDR format (e.g. 203.0.113.1/32).
    Type: String
    MinLength: '9'
    MaxLength: '18'
    Default: 0.0.0.0/0
    AllowedPattern: '^(\d{1,3}\.){3}\d{1,3}\/\d{1,2}$'
    ConstraintDescription: must be a valid IP CIDR range of the form x.x.x.x/x.

Resources:
  WebServerSecurityGroup:
    Type: AWS::EC2::SecurityGroup
    Properties:
      GroupDescription: Allow HTTP access via my IP address
      SecurityGroupIngress:
        - IpProtocol: tcp
          FromPort: 80
          ToPort: 80
          CidrIp: !Ref MyIP

  WebServer:
    Type: AWS::EC2::Instance
    Properties:
      ImageId: !Ref LatestAmiId
      InstanceType: !Ref InstanceType
      SecurityGroupIds:
        - !Ref WebServerSecurityGroup
      UserData: !Base64 |
        #!/bin/bash
        yum update -y
        yum install -y httpd
        systemctl start httpd
        systemctl enable httpd
        echo "<html><body><h1>Hello World!</h1></body></html>" > /var/www/html/index.html

Outputs:
  WebsiteURL:
    Value: !Join
      - ''
      - - http://
        - !GetAtt WebServer.PublicDnsName
    Description: Website URL

Antes de passar para a próxima etapa, vamos examinar o modelo e entender alguns conceitos-chave do Lambda.

A seção Parameters declara os valores que podem ser passados para o modelo quando você cria a pilha. Os recursos especificados posteriormente nesse modelo fazem referência a esses valores e usam os dados. Parâmetros são uma maneira eficaz de especificar informações que você não deseja armazenar no modelo em si. Também são uma maneira de especificar informações que sejam exclusivas da aplicação ou da configuração que você estiver implantando.

O modelo define os seguintes parâmetros:

LatestAmiId: recupera o ID do Amazon Linux 2 AMI mais recente da AWS Systems Manager Parameter Store.

InstanceType: permite a seleção do tipo de instância do EC2 (padrão: t2.micro, permitido: t3.micro, t2.micro).

MyIP: especifica o intervalo de endereços IP para acesso por HTTP (padrão: 0.0.0.0/0, permitindo acesso de qualquer IP).

A seção Resources contém as definições dos recursos da AWS que você deseja criar com o modelo. As declarações de recursos são uma maneira eficiente de especificar todas essas definições da configuração de uma só vez. Quando você inclui declarações de recursos em um modelo, pode criar e configurar todos os recursos declarados usando esse modelo para criar uma pilha. Você também pode criar novas pilhas a partir do mesmo modelo para iniciar configurações de recursos idênticas.

Este modelo cria os seguintes recursos:

WebServerSecurityGroup: um grupo de segurança do EC2 que permite tráfego HTTP de entrada na porta 80 originado no intervalo de IP especificado.

WebServer: uma instância do EC2 com a seguinte configuração:

Usa o Amazon Linux 2 AMI mais recente

Aplica o tipo de instância selecionado

Adiciona o WebServerSecurityGroup à propriedade SecurityGroupIds

Inclui um script de dados de usuário para instalar o Apache HTTP Server

Um nome lógico é especificado no início de cada declaração de recurso e parâmetro. Por exemplo, WebServerSecurityGroup é o nome lógico atribuído ao recurso do grupo de segurança do EC2. A função Ref é usada então para referenciar recursos e parâmetros por seus nomes lógicos em outras partes do modelo. Quando um recurso referencia outro, isso cria uma dependência entre eles.

A seção Outputs define valores personalizados que são retornados após a criação da pilha. Você pode usar os valores de saída para retornar informações dos recursos na pilha, como os identificadores ou URLs dos recursos.

O modelo define uma única saída:

WebsiteURL: a URL do servidor Web implantado, construída usando o nome de DNS público da instância EC2. A função Join ajuda a combinar o http:// fixo com o PublicDnsName variável em uma única string, facilitando a saída da URL completa do servidor Web.

Escolha Validar para se certificar de que o código YAML seja válido antes de carregar o modelo.

Depois, escolha Criar modelo para criar o modelo e adicioná-lo a um bucket do S3.

Na caixa de diálogo que se abre, anote o nome do bucket do S3 para poder excluí-lo posteriormente. Depois, escolha Confirmar e continuar no CloudFormation. Isso leva você para o console do CloudFormation, onde o caminho do S3 para o modelo agora está especificado.

Na página Criar pilha, selecione Avançar.

Na página Especificar detalhes da tarefa, digite um nome no campo Nome da pilha. O nome da pilha não pode conter espaços. Neste exemplo, use MyTestStack.

Em Parâmetros, especifique os valores dos parâmetros da seguinte maneira:

LatestAmiId: isso é definido por padrão como o Amazon Linux 2 AMI mais recente.

InstanceType: escolha t2.micro ou t3.micro como o tipo de instância do EC2.

MyIP: especifique seu endereço IP público real com um sufixo /32. O sufixo /32 é usado na notação CIDR para especificar que apenas um endereço IP é permitido. Isso significa, em essência, permitir tráfego de e para esse endereço IP específico, e nenhum outro.

Escolha Avançar duas vezes para ir até a página Revisar e criar. Para este tutorial, você pode deixar os padrões na página Configurar opções da pilha como estão.

Revise as informações da pilha. Quando estiver satisfeito com as configurações, clique em Submit (Enviar).